# Executors and Tasks

This notebook has a brief explanation of how we are going to create generic **Classes** capable of handling important Data Lake jobs. *E.g.*, transferring data from different storages, creating tables on **AWS Athena** and **AWS Redshift**, managing **Glue Jobs**, *etc*.

## Staging Data with a optional transformation

The class we are creating below is capable of taking data from a parent directory and moving it to a dump directory. Furthermore, given a python script, it will run it with its respective arguments.

In [74]:
class StagingExecutor:
    
    def __init__(self, parent_directory: str, dump_directory: str, archive_or_delete: str = "archive", py_exec_path: str = None, py_exec_args: dict = None) -> None:
        self.parent = parent_directory
        self.dump = dump_directory
        self.archive_or_delete = archive_or_delete
        self.py_exec_path = py_exec_path
        self.py_exec_args = py_exec_args

    def transfer(self) -> None:
        
        if not self.py_exec_path:
            print('No transformation required. Moving file using only parent and dump')
        
        elif self.py_exec_path:
            import sys
            sys.path.insert(1, self.py_exec_path)
            import py_exec
            
            py_exec.test()
            
            if not self.py_exec_args:
                print('Python Executor does not require arguments')
            
            elif self.py_exec_args:
                print(f'Python Executor is running with the following parameters:\n{self.py_exec_args}')
                py_exec.main(self.parent, self.dump, **self.py_exec_args)
                
    def post_staging(self) -> None:
                
        if self.archive_or_delete == 'archive':
            print('File from landing will be moved to archive folder')
        
        elif self.archive_or_delete == 'delete':
            print('File will be deleted from landing')

In [70]:
params = {'parent_directory': r'/home/corbanez/Documents/Data Engineering using AWS/data/raw_data.csv', \
            'dump_directory': r'/home/corbanez/Documents/Data Engineering using AWS/data/clean_data.csv', \
                #'archive_or_delete': 'delete', \
                     'py_exec_path': r'../executors', \
                          'py_exec_args': {'mapper': {'column1':'name', 'column2':'age', 'column3':'job'}}}

task = StagingExecutor(**params)

In [71]:
task.transfer()

task.post_staging()

Test ok
Python Executor is running with the following parameters:
{'mapper': {'column1': 'name', 'column2': 'age', 'column3': 'job'}}
File from landing will be moved to archive folder


In [73]:
routine_config = {'routine_name': 'Test Ingestion with py_exec', \
                    'tasks': {'StagingExecutor': {'params': params, \
                                                     'sub_tasks': ['transfer', 'post_staging']}}

for method in routine_config['sub_tasks']:
    getattr(task, method)()

Test ok
Python Executor is running with the following parameters:
{'mapper': {'column1': 'name', 'column2': 'age', 'column3': 'job'}}
File from landing will be moved to archive folder
